<a href="https://colab.research.google.com/github/gkalwaysno1/IEEE_OSPI/blob/main/Ensemble_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from keras.models import load_model
data_dir='/content/drive/MyDrive/Ensemble_dataset'
all_labels=['Brain_tumor','Skin']
image_list,label_list=[],[]
bin_labl=[0,1]
temp=-1

In [4]:
tf.data.Dataset.list_files

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.list_files(file_pattern, shuffle=None, seed=None, name=None)>

In [5]:
data=tf.keras.utils.image_dataset_from_directory(data_dir)

Found 1751 files belonging to 2 classes.


In [6]:
data_iterator=data.as_numpy_iterator()
data_iterator

In [7]:
batch=data_iterator.next()
batch[1]

array([1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1], dtype=int32)

In [8]:
def convert_image_to_array(image_path):
    image = Image.open(image_path)
    image_array = np.array(image)
    return image_array

In [9]:
for dir in all_labels:
  brn_img_list=os.listdir(f"{data_dir}/{dir}/")
  temp+=1
  for files in brn_img_list:
    try:
      img_path=f"{data_dir}/{dir}/{files}"
      image_list.append(convert_image_to_array(img_path))
      label_list.append(bin_labl[temp])
    except:
        print(f"Error opening image at {img_path}")

In [10]:
import pandas as pd
label_counts=pd.DataFrame(label_list).value_counts()
label_counts.head()

0    1012
1     739
dtype: int64

In [11]:
# def resize_images_in_folder(folder_path, target_size=(256, 256)):
#     for filename in os.listdir(folder_path):
#         img_path = os.path.join(folder_path, filename)

#         try:
#             img = Image.open(img_path)

#             # Convert image to RGB mode if it's in the 'P' mode
#             if img.mode != 'RGB':
#                 img = img.convert('RGB')

#             img_resized = img.resize(target_size)
#             img_resized.save(img_path)
#         except Exception as e:
#             print(f"Error processing {img_path}: {e}")

# # Replace 'folder1', 'folder2', 'folder3', and 'folder4' with your actual folder paths
# folders = ['/content/drive/MyDrive/Ensemble_dataset/Brain_tumor',
#            '/content/drive/MyDrive/Ensemble_dataset/Skin']

# for folder in folders:
#     resize_images_in_folder(folder)

In [12]:
image_list[0].shape

(256, 256, 3)

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(image_list,label_list,test_size=0.2,random_state=10)


In [14]:
x_train=np.array(x_train,dtype=np.float16)/255.0
x_test=np.array(x_test,dtype=np.float16)/255.0
x_train=x_train.reshape(-1,256,256,3)
x_test=x_test.reshape(-1,256,256,3)

In [15]:
from keras.utils import to_categorical
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [16]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
model=Sequential()
model.add(Conv2D(32,(3,3),padding="same",input_shape=(256,256,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(16,(3,3),padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(8,activation="relu"))
model.add(Dense(4,activation="softmax"))
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 85, 85, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 85, 85, 16)        4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 42, 42, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 28224)             0         
                                                                 
 dense (Dense)               (None, 8)                 2

In [17]:
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=10)


In [18]:
history=model.fit(x_train,y_train,batch_size=128,epochs=18,validation_data=(x_val,y_val))

Epoch 1/18
9/9 [==============================] - 16s 407ms/step - loss: 0.6368 - accuracy: 0.7295 - val_loss: 0.5504 - val_accuracy: 0.9464
Epoch 2/18
9/9 [==============================] - 1s 141ms/step - loss: 0.5121 - accuracy: 0.9554 - val_loss: 0.4748 - val_accuracy: 0.9821
Epoch 3/18
9/9 [==============================] - 1s 143ms/step - loss: 0.4598 - accuracy: 0.9768 - val_loss: 0.4592 - val_accuracy: 0.9571
Epoch 4/18
9/9 [==============================] - 2s 173ms/step - loss: 0.4489 - accuracy: 0.9750 - val_loss: 0.4454 - val_accuracy: 0.9643
Epoch 5/18
9/9 [==============================] - 1s 169ms/step - loss: 0.4386 - accuracy: 0.9804 - val_loss: 0.4315 - val_accuracy: 0.9857
Epoch 6/18
9/9 [==============================] - 2s 174ms/step - loss: 0.4299 - accuracy: 0.9848 - val_loss: 0.4268 - val_accuracy: 0.9857
Epoch 7/18
9/9 [==============================] - 1s 143ms/step - loss: 0.4245 - accuracy: 0.9866 - val_loss: 0.4243 - val_accuracy: 0.9857
Epoch 8/18
9/9 [===

In [19]:
model.save("/content/drive/MyDrive/Ensemble_dataset/Model/ens.h5")